In [ ]:
import numpy as np
from pathlib import Path
from kalman_reconstruction.custom_plot import (
    set_custom_rcParams,
    adjust_lightness,
    handler_map_alpha,
    plot_colors,
    symmetrize_axis,
)
from reconstruct_climate_indices.idealized_ocean import spunge_ocean, oscillatory_ocean
from reconstruct_climate_indices.statistics import linear_regression_loglog
import matplotlib.pyplot as plt
from kalman_reconstruction.statistics import normalize

# from sklearn.linear_model import LinearRegression
from scipy import signal

In [ ]:
150 * 365.25 / 30.4375

1800.0

In [ ]:
# plt.style.use('default')
# dark_color = [0.2, 0.2, 0.2]
# light_color = [0.9, 0.9, 0.9]
# lightness_0 = 0.75
# lightness_1 = 0.5
plt.style.use("dark_background")
dark_color = [0.7, 0.7, 0.7]
light_color = [0.1, 0.1, 0.1]
lightness_0 = 1.15
lightness_1 = 1.5
set_custom_rcParams()
plt.rcParams["axes.grid"] = False
colors = [
    "#CC6677",
    "#6E9CB3",
    "#CA8727",
    "#44AA99",
    "#AA4499",
    "#D6BE49",
    "#A494F5",
]
plot_colors(colors)
variables_color = dict()
variables_color["SAT"] = colors[0]
variables_color["SST"] = colors[2]
variables_color["DOT"] = colors[1]
variables_color["latent"] = colors[3]

In [ ]:
REPO_PATH = Path(".").resolve().parent
results_path = REPO_PATH / Path("results") / "Report" / "data" / "idealized_ocean"
results_path.mkdir(parents=True, exist_ok=True)
SAVE_FIGURES = True


def save_fig(fig, relative_path, **kwargs):
    store_path = results_path / relative_path
    store_path.parent.mkdir(parents=True, exist_ok=True)
    if SAVE_FIGURES:
        fig.savefig(store_path, **kwargs)
    else:
        pass

In [ ]:
# 50 years
dt = 30
years = 1000
time_steps = years * 365.25 / dt

tau0 = 10  # years
lambda0 = np.pi * tau0 / 2  # years
omega0 = 24  # years
fs = 365.25 / dt  # 1/years

welch_window_width = 250  # years

spunge_settings = dict(
    nt=time_steps,
    dt=dt,
    df=1.15e-1,
    tau0=tau0 * 365.25,
    save_path=None,
)
spunge_data = spunge_ocean(**spunge_settings)
# display(data)
oscillatory_settings = dict(
    nt=time_steps,
    dt=dt,
    df=1.15e-1,
    tau0=tau0 * 365.25,
    per0=omega0 * 365.25,
    save_path=None,
)
oscillatory_data = oscillatory_ocean(**oscillatory_settings)

print(rf"$\lambda_0$ = {lambda0}")
print(rf"$\omega_0$ = {omega0}")

$\lambda_0$ = 15.707963267948966
$\omega_0$ = 24


C:\Users\Niebaum\Documents\Repositories\reconstruct-climate-indices\reconstruct_climate_indices\idealized_ocean.py:52: UserWarning: The given timesteps were not in int format.
They were modified and are now: timesteps = 12175
  warn(


### Check the noramlity of the data

In [ ]:
fig, axs = plt.subplots(
    nrows=1, ncols=2, layout="constrained", sharex=True, sharey=True
)

stepfill_kwargs = dict(
    bins=31,
    histtype="stepfilled",
    density=True,
    alpha=0.1,
)
step_kwargs = dict(
    bins=31,
    histtype="step",
    density=True,
    alpha=1,
    linewidth=3,
)

axs_spunge = axs[0]
axs_oscill = axs[1]
axs_spunge.set_title("Sponge Ocean")
axs_oscill.set_title("Oscillatory Ocean")

# Plot Spunge distribution
for var in ["SAT", "SST"]:
    his = axs_spunge.hist(
        normalize(spunge_data[var], method="mean"),
        label=var,
        color=variables_color[var],
        **step_kwargs,
    )
    axs_spunge.hist(
        normalize(spunge_data[var], method="mean"),
        color=variables_color[var],
        **stepfill_kwargs,
    )

# Plot Oscilltor distribution
for var in ["SAT", "SST", "DOT"]:
    his = axs_oscill.hist(
        normalize(oscillatory_data[var], method="mean"),
        label=var,
        color=variables_color[var],
        **step_kwargs,
    )
    axs_oscill.hist(
        normalize(oscillatory_data[var], method="mean"),
        color=variables_color[var],
        **stepfill_kwargs,
    )

# Plot normal distribution
import scipy.stats as stats

mu = 0
variance = 1
width = 4
sigma = np.sqrt(variance)
x = np.linspace(mu - width * sigma, mu + width * sigma, 100)
for ax in axs.flatten():
    ax.plot(
        x,
        stats.norm.pdf(x, mu, sigma),
        color=dark_color,
        label=r"$\mathcal{N}(0,1)$",
        linewidth=4,
    )
    ax.legend()
    ax.set_xlabel("Normalized Values")
    symmetrize_axis(axes=ax, axis="x")
axs[0].set_ylabel("Probability Density")

save_fig(fig=fig, relative_path=f"Distribution_{years}y.pdf")

### Plot the evolution for the firsr 50 yeras

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, layout="constrained", sharex=True)
kwargs = dict(
    linestyle="-",
    linewidth="1",
    marker=".",
)

idx = 0  # x index to plot the text at

axs_spunge = axs[:, 0]
axs_oscill = axs[:, 1]
axs_spunge[0].set_title("Sponge Ocean")
axs_oscill[0].set_title("Oscillatory Ocean")

# -------------------
# Plot SPONGE OCEAN
# -------------------

# Plot SAT
var = "SAT"
axs_spunge[0].plot(
    spunge_data["time_years"],
    spunge_data[var],
    label=var,
    color=variables_color[var],
    **kwargs,
)
axs_spunge[0].legend()
axs_spunge[0].set_ylabel(r"$y^{0.5}$")

# Plot SST
var = "SST"
axs_spunge[1].plot(
    spunge_data["time_years"],
    spunge_data[var],
    label=var,
    color=variables_color[var],
    **kwargs,
)
axs_spunge[1].legend()
axs_spunge[1].set_ylabel(r"K")

# -------------------
# Plot Oscillatory OCEAN
# -------------------
var = "SAT"
axs_oscill[0].plot(
    oscillatory_data["time_years"],
    oscillatory_data[var],
    label=var,
    color=variables_color[var],
    **kwargs,
)
axs_oscill[0].set_ylabel(r"K $y^{0.5}$")
axs_oscill[0].legend()

for var in ["SST", "DOT"]:
    axs_oscill[1].plot(
        oscillatory_data["time_years"],
        oscillatory_data[var],
        label=var,
        color=variables_color[var],
        **kwargs,
    )
axs_oscill[1].legend()
axs_oscill[1].set_ylabel(r"K")


axs_oscill[0].set_ylim([-0.5, 0.5])
axs_spunge[0].set_ylim([-0.5, 0.5])

axs_oscill[1].set_ylim([-12, 12])
axs_spunge[1].set_ylim([-12, 12])

for i in range(2):
    axs_spunge[i].set_xlim([0, 50])
    axs_oscill[i].set_xlim([0, 50])


save_fig(fig=fig, relative_path=f"Evolution_{years}y.pdf")

### Plot the Frequency analysis

In [ ]:
fig, axs = plt.subplots(
    nrows=2, ncols=2, layout="constrained", sharex=True, sharey=True
)
kwargs = dict(
    linestyle="-",
    linewidth="1",
    marker=".",
)

idx = 0  # x index to plot the text at

axs_spunge = axs[:, 0]
axs_oscill = axs[:, 1]
axs_spunge[0].set_title("Sponge Ocean")
axs_oscill[0].set_title("Oscillatory Ocean")

# ------------------
# plot SAT
# ------------------
var = "SAT"
for d, ax in zip([spunge_data, oscillatory_data], [axs_spunge[0], axs_oscill[0]]):
    color = variables_color[var]
    x = d[var].values
    frequencies, spectrum = signal.welch(x=x, fs=fs, window="hann", nperseg=len(x))
    ax.loglog(frequencies, spectrum, label=f"{var}", color=color, alpha=0.7)
    frequencies, spectrum = signal.welch(
        x=x, fs=fs, window="hann", nperseg=int(welch_window_width * fs)
    )
    ax.loglog(
        frequencies,
        spectrum,
        label=f"{var} welch",
        color=adjust_lightness(color, lightness_0),
    )
    frequencies_linear, spectrum_linear, regression = linear_regression_loglog(
        frequencies=frequencies, spectrum=spectrum, weights="f_inverse"
    )
    slope = regression.coef_[0, 0]
    ax.loglog(
        frequencies_linear,
        spectrum_linear,
        color=adjust_lightness(color, lightness_1),
    )
    ax.text(
        frequencies_linear[idx],
        spectrum_linear[idx],
        f"m={slope:.2f}",
        ha="right",
        va="bottom",
        bbox=dict(facecolor=light_color, edgecolor="None", alpha=0.25),
        color=adjust_lightness(color, lightness_1),
    )


# ------------------
# plot SST
# ------------------
var = "SST"
for d, ax, f_low in zip(
    [spunge_data, oscillatory_data],  # datasets
    [axs_spunge[1], axs_oscill[1]],  # axes object to plot into
    [1 / lambda0, 1 / omega0],  # lowest frequency to use for linear regression
):
    x = d[var].values
    color = variables_color[var]
    frequencies, spectrum = signal.welch(x=x, fs=fs, window="hann", nperseg=len(x))
    h = ax.loglog(
        frequencies,
        spectrum,
        label=f"{var}",
        color=color,
        alpha=0.7,
    )
    frequencies, spectrum = signal.welch(
        x=x, fs=fs, window="hann", nperseg=int(welch_window_width * fs)
    )
    ax.loglog(
        frequencies,
        spectrum,
        label=f"{var} welch",
        color=adjust_lightness(color, lightness_0),
    )
    frequencies_linear, spectrum_linear, regression = linear_regression_loglog(
        frequencies=frequencies, spectrum=spectrum, f_low=f_low, weights="f_inverse"
    )
    slope = regression.coef_[0, 0]
    ax.loglog(
        frequencies_linear,
        spectrum_linear,
        color=adjust_lightness(color, lightness_1),
    )
    ax.text(
        frequencies_linear[idx],
        spectrum_linear[idx],
        f"m={slope:.2f}",
        ha="right",
        va="bottom",
        bbox=dict(facecolor=light_color, edgecolor="None", alpha=0.25),
        color=adjust_lightness(color, lightness_1),
    )


# ------------------
# plot DOT
# ------------------
var = "DOT"
for d, ax, f_low in zip(
    [oscillatory_data],  # datasets
    [axs_oscill[1]],  # axes object to plot into
    [1 / omega0],  # lowest frequency to use for linear regression
):
    color = variables_color[var]
    x = d[var].values
    frequencies, spectrum = signal.welch(x=x, fs=fs, window="hann", nperseg=len(x))
    h = ax.loglog(
        frequencies,
        spectrum,
        label=f"{var}",
        alpha=0.7,
        color=color,
    )
    frequencies, spectrum = signal.welch(
        x=x, fs=fs, window="hann", nperseg=int(welch_window_width * fs)
    )
    ax.loglog(
        frequencies,
        spectrum,
        label=f"{var} welch",
        color=adjust_lightness(color, lightness_0),
    )
    frequencies_linear, spectrum_linear, regression = linear_regression_loglog(
        frequencies=frequencies, spectrum=spectrum, f_low=f_low, weights="f_inverse"
    )
    slope = regression.coef_[0, 0]
    ax.loglog(
        frequencies_linear,
        spectrum_linear,
        color=adjust_lightness(color, lightness_1),
    )
    ax.text(
        frequencies_linear[idx],
        spectrum_linear[idx],
        f"m={slope:.2f}",
        ha="right",
        va="bottom",
        bbox=dict(facecolor=light_color, edgecolor="None", alpha=0.25),
        color=adjust_lightness(color, lightness_1),
    )


for ax in axs.flatten():
    ax.axvline(1 / lambda0, color=dark_color, linestyle=":")  # , label = fr"$\tau_0$")
    ax.text(
        1 / lambda0,
        10 ** (-6.5),
        r"$\pi/(2\tau_0)$",
        ha="left",
        va="bottom",
        c=dark_color,
        rotation=90,
    )
    ax.axvline(
        1 / omega0, color=dark_color, linestyle="--"
    )  # , label = fr"$\omega_0$")
    ax.text(
        1 / omega0,
        10 ** (-6.5),
        r"1/$\omega_0$",
        ha="right",
        va="bottom",
        c=dark_color,
        rotation=90,
    )
    ax.axvline(
        1 / welch_window_width, color=dark_color, linestyle="-"
    )  # , label = f"welch window.:\n{welch_window_width}")
    ax.text(
        1 / welch_window_width,
        10 ** (-6.5),
        r"$f_{welch}$",
        ha="right",
        va="bottom",
        c=dark_color,
        rotation=90,
    )
    ax.legend(ncol=2, labelspacing=0.01, loc="lower left")

# fig.suptitle(f"$\tau_0$:{tau0} years | $\omega_0$:{omega0} years | welch_windon:{welch_window_width} years | duration {years} years")

save_fig(fig=fig, relative_path=f"Frequency_{years}y.pdf")